<a href="https://colab.research.google.com/github/WISEPLAT/_NN/blob/main/%D0%9F%D1%80%D0%BE%D0%B2%D0%B5%D1%80%D0%BA%D0%B0_%D0%93%D0%B8%D0%BF%D0%BE%D1%82%D0%B5%D0%B7%D1%8B_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h516909a_0.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py37ha21ca33_2.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.7/dist-packages/ lib/python3.7/site-packages/talib --strip-components=3
import talib
from talib import abstract

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3689    0  3689    0     0  17650      0 --:--:-- --:--:-- --:--:-- 17735
100  503k  100  503k    0     0   909k      0 --:--:-- --:--:-- --:--:--  909k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3709    0  3709    0     0  18270      0 --:--:-- --:--:-- --:--:-- 18181
100  406k  100  406k    0     0   840k      0 --:--:-- --:--:-- --:--:--  840k


In [3]:
# daily prices from github
url = ('https://raw.githubusercontent.com/WISEPLAT/financial-data/main/GMKN_000101_211021.csv')
df = pd.read_csv(url, sep=";")
print(df.shape, df.columns)
df.rename(columns={"<DATE>":"date","<TIME>":"time", "<OPEN>":"open","<HIGH>":"high", "<LOW>":"low", "<CLOSE>":"close","<VOL>":"volume"}, inplace=True)
print(df.shape, df.columns)
df['date']=pd.to_datetime(df['date'], format='%Y%m%d')
df=df.drop('time', 1)
print(df.shape, df.columns)

(4997, 7) Index(['<DATE>', '<TIME>', '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<VOL>'], dtype='object')
(4997, 7) Index(['date', 'time', 'open', 'high', 'low', 'close', 'volume'], dtype='object')
(4997, 6) Index(['date', 'open', 'high', 'low', 'close', 'volume'], dtype='object')


In [4]:
df

,date,open,high,low,close,volume
0,2001-10-31,379.00,379.00,366.00,366.01,1200
1,2001-11-01,371.99,378.66,371.99,378.45,7725
2,2001-11-02,380.01,382.00,362.00,365.00,21666
3,2001-11-05,364.00,373.50,358.00,365.00,44998
4,2001-11-06,367.00,370.00,350.00,355.00,10174
...,...,...,...,...,...,...
4992,2021-10-15,22950.00,23124.00,22762.00,23108.00,244687
4993,2021-10-18,23122.00,23178.00,22974.00,23120.00,151319
4994,2021-10-19,23200.00,23700.00,23166.00,23672.00,344539
4995,2021-10-20,23680.00,23698.00,23190.00,23530.00,263589


In [5]:
print(df["date"].min(), df["date"].max(), df.shape)

2001-10-31 00:00:00 2021-10-21 00:00:00 (4997, 6)


In [6]:
#df01 - берем среднюю цену инструмента
df01_middle = np.array((df["open"]+df["close"])/2)

df01_high_minus_low = np.array(df["high"]-df["low"])
df01_intraday_price_change_in_percent = np.array(df["close"]/df["open"]-1) #np.array(df["close"]*100/df["open"]-100)

# берем "вчерашнюю" цену закрытия и "сегодняшнюю" цену закрытия - для учитывания взаимосвязи предыдущего дня
df01_percent = np.array(df["close"]*100/df["close"].shift()-100)

In [7]:
df01_percent

array([        nan,  3.39881424, -3.55397014, ...,  2.38754325,
       -0.59986482, -1.359966  ])

In [8]:
#df02 - daily volume change in %
# берем "вчерашний" объем и "сегодняшний" объем - для учитывания взаимосвязи предыдущего дня
df02 = np.array(df["volume"]*100/df["volume"].shift()-100)

In [9]:
df02

array([         nan, 543.75      , 180.46601942, ..., 127.69050813,
       -23.4951631 , -36.65593026])

In [10]:
df02.shape[0]

4997

In [11]:
def calc_up_down(df):
  _up = 0; df_up = []; _down = 0; df_down = [];
  for i in range(1, df.shape[0]):
    #calc _up
    if df[i]>=df[i-1]:
      _up += 1
    else:
      _up = 0
    df_up = np.append(df_up, _up)
    #calc _down
    if df[i]<=df[i-1]:
      _down += 1
    else:
      _down = 0
    df_down = np.append(df_down, _down)
    #print(_up, end=" ")
    df_up = np.array(df_up)
    df_down = np.array(df_down)
  return df_up, df_down

df02_up, df02_down = calc_up_down(df02)


In [12]:
df02_up

array([0., 0., 0., ..., 1., 0., 0.])

In [13]:
df02_down

array([0., 1., 2., ..., 0., 1., 2.])

In [14]:
#df03 - MACD
_fastperiod=12; _slowperiod=26; _signalperiod=9
ms = abstract.MACD(df, fastperiod=_fastperiod, slowperiod=_slowperiod, signalperiod=_signalperiod)

In [15]:
ms

,macd,macdsignal,macdhist
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
4992,-232.027218,-402.360157,170.332939
4993,-175.370046,-356.962135,181.592089
4994,-84.947817,-302.559271,217.611454
4995,-24.463759,-246.940169,222.476409


In [16]:
df03 = np.array(ms['macd'])

In [17]:
df03

array([         nan,          nan,          nan, ..., -84.94781699,
       -24.46375924,  -2.32424095])

In [18]:
df03_up, df03_down = calc_up_down(df03)

In [19]:
df03_up

array([ 0.,  0.,  0., ...,  9., 10., 11.])

In [20]:
df03_down

array([0., 0., 0., ..., 0., 0., 0.])

In [21]:
#df04 - Relative Strength Index (Momentum Indicators)
_period_indicator = 50
ms = abstract.RSI(df, timeperiod=_period_indicator)
df04 = np.array(ms)

In [22]:
df04

array([        nan,         nan,         nan, ..., 48.96916331,
       48.463264  , 47.33867695])

In [23]:
df04_up, df04_down = calc_up_down(df04)

In [24]:
df04_up

array([0., 0., 0., ..., 5., 0., 0.])

In [25]:
df04_down

array([0., 0., 0., ..., 0., 1., 2.])

In [26]:
#df06 - Money Flow Index (Momentum Indicators)
_period_indicator = 14
ms = abstract.MFI(df, timeperiod=_period_indicator)
df06 = np.array(ms)

In [27]:
df06

array([        nan,         nan,         nan, ..., 66.40019543,
       68.56538338, 63.97733899])

In [28]:
df06_up, df06_down = calc_up_down(df06)

In [29]:
df06_up

array([0., 0., 0., ..., 4., 5., 0.])

In [30]:
df06_down

array([0., 0., 0., ..., 0., 0., 1.])

In [31]:
#df10 - Average True Range (Volatility Indicators)
_period_indicator = 14
ms = abstract.ATR(df, timeperiod=_period_indicator)
df10 = np.array(ms)

In [32]:
df10

array([         nan,          nan,          nan, ..., 504.17849772,
       504.45146217, 495.70492916])

In [33]:
df10_up, df10_down = calc_up_down(df10)

In [34]:
df10_up

array([0., 0., 0., ..., 1., 2., 0.])

In [35]:
df10_down

array([0., 0., 0., ..., 0., 0., 1.])

In [36]:
#Price% df01_percent                
#Volume% df02_up, df02_down
#MACD df03_up, df03_down
#RSI df04_up, df04_down
#MFI df06_up, df06_down
#ATR df10_up, df10_down

In [37]:
from datetime import timedelta
data_1_yr = df[df["date"] > df["date"].max() - timedelta(days=365)]

In [38]:
data_1_yr

,date,open,high,low,close,volume
4742,2020-10-22,19788.0,19850.0,19504.0,19672.0,251474
4743,2020-10-23,19716.0,19966.0,19540.0,19838.0,198802
4744,2020-10-26,19780.0,19838.0,19420.0,19514.0,280332
4745,2020-10-27,19512.0,19526.0,19100.0,19150.0,293221
4746,2020-10-28,19104.0,19210.0,18400.0,18548.0,377170
...,...,...,...,...,...,...
4992,2021-10-15,22950.0,23124.0,22762.0,23108.0,244687
4993,2021-10-18,23122.0,23178.0,22974.0,23120.0,151319
4994,2021-10-19,23200.0,23700.0,23166.0,23672.0,344539
4995,2021-10-20,23680.0,23698.0,23190.0,23530.0,263589


In [39]:
train_size = int(data_1_yr.shape[0]*0.8)
train_data = data_1_yr.iloc[:train_size]
validate_data = data_1_yr.iloc[train_size:]

In [40]:
train_data.shape, validate_data.shape

((204, 6), (51, 6))

In [41]:
train_data["date"].min(), train_data["date"].max()

(Timestamp('2020-10-22 00:00:00'), Timestamp('2021-08-11 00:00:00'))

In [42]:
validate_data["date"].min(), validate_data["date"].max()

(Timestamp('2021-08-12 00:00:00'), Timestamp('2021-10-21 00:00:00'))

In [43]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

scaler = StandardScaler()
scaler.fit(train_data[["close"]])

def make_dataset(
    df,                 #данные для создания датасета
    window_size,        #кол. элементов для предсказания след. элемента
    batch_size,         #кол. элементов в batch для обучения
    use_scaler=True,    #использовать ли Scaler для нормализации свечей
    shuffle=True        #смешивать элементы в датасете или нет
):
  features = df[["close"]].iloc[:-window_size]      #-N предыдущих элементов как фичи, поэтому вычитаем
  print(features)
  print(df["close"].iloc[window_size:])
  if use_scaler:                                  #нормализуем фичи, если нужно
    features=scaler.transform(features)
  data = np.array(features, dtype=np.float32)     #приводим данные к нужному типу
  ds = tf.keras.preprocessing.timeseries_dataset_from_array(    #создание датасета с временным рядом для обучения НС
      data=data,                                  #принимает многомерный массив, кот. содержит фичи
      targets=df["close"].iloc[window_size:],       #принимает лейблы, которые нужно сдвинуть на +N элементов
                                                  #т.е. для каждых N элементов из фичей будем иметь N+1 элемент как лейбл
      sequence_length=window_size,                #принимает длину нашей последовательности window_size
      sequence_stride=1,                          #на сколько нужно сдвигать элемент для создания нового семпла, т.е. предсказываем каждый след. элемент
      shuffle=shuffle,                            #нужно ли перемешивать элементы
      batch_size=batch_size                       #какой размер батча
  )
  return ds


In [44]:
example_ds = make_dataset(df=train_data, window_size=3, batch_size=2, use_scaler=False, shuffle=False)

        close
4742  19672.0
4743  19838.0
4744  19514.0
4745  19150.0
4746  18548.0
...       ...
4938  25290.0
4939  25582.0
4940  25446.0
4941  25496.0
4942  25102.0

[201 rows x 1 columns]
4745    19150.0
4746    18548.0
4747    18998.0
4748    18864.0
4749    19334.0
         ...   
4941    25496.0
4942    25102.0
4943    24870.0
4944    24650.0
4945    24528.0
Name: close, Length: 201, dtype: float64


In [45]:
example_feature, example_label = next(example_ds.as_numpy_iterator())

In [46]:
example_feature.shape

(2, 3, 1)

In [47]:
example_label.shape

(2,)

In [48]:
train_data["close"].iloc[:6]

4742    19672.0
4743    19838.0
4744    19514.0
4745    19150.0
4746    18548.0
4747    18998.0
Name: close, dtype: float64

In [49]:
print(example_feature[0])
print(example_label[0])

[[19672.]
 [19838.]
 [19514.]]
19150.0


In [50]:
print(example_feature[1])
print(example_label[1])

[[19838.]
 [19514.]
 [19150.]]
18548.0
